In [ ]:
!pip install langchain
!pip install youtube-transcript-api
!pip install google-generativeai
!pip install litellm
!pip install sentence_transformers
!pip install chromadb

In [ ]:
import re
import json
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

## Get YouTube video URL

In [ ]:
def get_video_transcript():
  video_id = input("Enter YouTube video ID: ")
  transcript = YouTubeTranscriptApi.get_transcript(video_id)
  # Convert Python list of dictionaries to a JSON string
  transcript = json.dumps(transcript)
  # Parse the JSON string back into a Python list of dictionaries
  transcript = json.loads(transcript)
  # Combine the text values from the list of dictionaries
  combinedText = ' '.join((obj['text']+"\n") for obj in transcript).strip()
  return combinedText


video_transcript = get_video_transcript()



## Chunk up video transcript into smaller documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_transcript = text_splitter.split_text(video_transcript)
texts = [Document(page_content=t) for t in split_transcript]

In [ ]:
print (f'Now you have {len(texts)} documents')

## Generate a summary using a chat model

In [ ]:
llm = ChatOpenAI(temperature=0, openai_api_key="YOUR_API_KEY")
chain = load_summarize_chain(llm, chain_type="refine")
chain.run(docs)